# Project 5: Optimizing Evacuation Routes using Real-Time Traffic Information

Song May, Michael Daugherty, Kelly Slatery | US-DSI-10 | 02.21.2020

## Get Coordinates

In [35]:
2*9*5*7*11*13*17*19*8


232792560

In [5]:

def fibonacci(n):
    n -= 2
    i = 1
    j = 1

    while n > 0:
        n -= 1
        num = i + j
        i = j
        j = num
    return num
    
fibonacci(7)

13

# Import lat/long data file

In [42]:
ll = pd.read_csv('./data/locations_w_LL.csv')
ll.shape

(43, 3)

In [43]:
ll.head()

,Unnamed: 0,location,latlng
0,0,"I-10 East at Airway, Texas","(31.77851238297871, -106.39274056382978)"
1,1,"I-10 East at Mesa, Texas","(31.76263800000001, -106.49052)"
2,2,"I-10 East at Asarco, Texas","(31.152728536211036, -97.45027245045877)"
3,3,"I-10 West at Lomaland, Texas","(31.744404000000007, -106.33187249999999)"
4,4,"I-10 East at Sunland, Texas","(31.816627713233906, -106.55121538620033)"


In [44]:
ll.drop(columns='Unnamed: 0', inplace=True)
ll.head()

,location,latlng
0,"I-10 East at Airway, Texas","(31.77851238297871, -106.39274056382978)"
1,"I-10 East at Mesa, Texas","(31.76263800000001, -106.49052)"
2,"I-10 East at Asarco, Texas","(31.152728536211036, -97.45027245045877)"
3,"I-10 West at Lomaland, Texas","(31.744404000000007, -106.33187249999999)"
4,"I-10 East at Sunland, Texas","(31.816627713233906, -106.55121538620033)"


In [48]:
# Split up latitude and longitude
lats = []
lngs = []

for coords in ll['latlng']:
    coords = coords.strip().replace('(', '').replace(')', '')
    lat, lng = coords.split()
    #print(lat.strip(','))
    lats.append(float(lat.strip(',')))
    lngs.append(float(lng))

# Verify that lists line up with dataframe values
print(lats[:5])
print(lngs[:5])

[31.77851238297871, 31.76263800000001, 31.152728536211036, 31.744404000000007, 31.816627713233906]
[-106.39274056382978, -106.49052, -97.45027245045877, -106.33187249999999, -106.55121538620033]


In [49]:
# Create new columns for lats and longs
ll['lat'] = lats
ll['lng'] = lngs

# Verify that columns were created
ll.head()

,location,latlng,lat,lng
0,"I-10 East at Airway, Texas","(31.77851238297871, -106.39274056382978)",31.778512,-106.392741
1,"I-10 East at Mesa, Texas","(31.76263800000001, -106.49052)",31.762638,-106.490520
2,"I-10 East at Asarco, Texas","(31.152728536211036, -97.45027245045877)",31.152729,-97.450272
3,"I-10 West at Lomaland, Texas","(31.744404000000007, -106.33187249999999)",31.744404,-106.331872
4,"I-10 East at Sunland, Texas","(31.816627713233906, -106.55121538620033)",31.816628,-106.551215


In [50]:
# Drop all non-lat/lng columns
ll.drop(columns=['location', 'latlng'], inplace=True)
print(ll.shape)
ll.head()

(43, 2)


,lat,lng
0,31.778512,-106.392741
1,31.762638,-106.490520
2,31.152729,-97.450272
3,31.744404,-106.331872
4,31.816628,-106.551215


In [51]:
# Check data types (float)
ll.dtypes

lat    float64
lng    float64
dtype: object

## Export Latitudes / Longitudes

In [52]:
ll.to_csv('./data/lat_lng.csv', index=False)

In [6]:
# Imports
import pandas as pd
import numpy as np
import re
import spacy

In [2]:
# Set view options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

# Import Data

In [3]:
full_df = pd.read_csv('./data/final_data_with_predictions.csv')
full_df.shape

(3087, 4)

In [4]:
full_df.head()

,dates&time,user,tweet,class
0,2017-08-31 21:34:27+00:00,DallasPD,"Reminder: Texas State Law prohibiting texting and driving goes into effect September 1, 2017",0
1,2017-08-31 21:03:18+00:00,DallasPD,Brotherhood for the Fallen Charity Event,0
2,2017-08-31 17:16:43+00:00,DallasPD,"FREE #TRINITY EVENT - Artisan Fair Market - September 2, 2017 #LaborDayWeekend",0
3,2017-08-31 17:13:36+00:00,DallasPD,Our thoughts and prayers are with the Sacramento County Sheriff's Department and the family of Deputy Sheriff Robert French.,0
4,2017-08-31 17:10:47+00:00,DallasPD,How can Dallas residents help animals affected by #HurricaneHarvey? http://fb.me/90nmndc7A,0


# Filter Data for Useful Tweets

In [5]:
# Filter df for useful tweets
df = full_df[full_df['class'] == 1]
df.shape

(103, 4)

In [6]:
df.head()

,dates&time,user,tweet,class
239,2017-09-01 04:36:40+00:00,EPPOLICE,"I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.",1
240,2017-09-01 04:09:53+00:00,EPPOLICE,"I-10 East at Mesa, schedule road work, all lanes closed, follow detour, clearing time 6 am.",1
242,2017-09-01 00:39:43+00:00,EPPOLICE,"I-10 East @US-54, Collision, Right Lane Closed, Backup To Porfirio Diaz, Clearing Time 1 Hour.",1
247,2017-08-31 23:10:23+00:00,EPPOLICE,"I-10 East @Schuster, Collision, Right Shoulder Closed, Backup To Executive, Clearing Time 1 Hour.",1
248,2017-08-31 22:51:34+00:00,EPPOLICE,"I-10 West @Trowbridge, Collision, Right Lane Closed, Backup To Hawkins, Clearing Time 1 Hour.",1


In [20]:
df.tail()

,dates&time,user,tweet,class
2914,2017-08-25 13:00:53+00:00,I35travel,Waxahachie | US 287 Business WB@I-35 E | Various lanes closed today 9AM to 3PM http://i35-maps.tti.tamu.edu/?id=6342 #My35,1
2948,2017-08-31 18:31:58+00:00,TxDOTLufkin,"ALERT: SL 287 East remains closed in Lufkin. Alternate routes are: SL 287 West, BU 59. From Nacogdoches, SH7 W to US 69 S to SL 287 West.",1
3040,2017-08-30 02:10:33+00:00,my290Houston,US 290 EB closed from Fry to Barker Cypress is closed. Only high clearance emergency vehicles can get through.,1
3053,2017-08-29 15:48:59+00:00,my290Houston,"No, if you do not need to travel, please stay put. US 290 EB is closed at Fry. Visit http://drivetexas.org to see updates.",1
3069,2017-08-28 13:59:02+00:00,my290Houston,US 290 mainlanes & frontage are closed in both directions from Barker Cypress to Fry Road due to high water.,1


# Create Location List

In [10]:
# Clean tweets: Map common abbreviations to full-length word
abbrevs = {
    'SB': 'southbound',
    'NB': 'northbound', 
    'EB': 'eastbound', 
    'WB': 'westbound', 
    'St': 'Street', 
    'Rd': 'Road', 
    'E.': 'east',
    'S.': 'south',
    'W.': 'west',
    'N.': 'north',
    'Blvd': 'boulevard',
    'Pkwy': 'parkway',
    'Hwy': 'highway',
    '@': ' at '  
}

# Map 
clean_tweets = []
for tweet in df['tweet']:
    for abbrev, word in abbrevs.items():
        tweet = tweet.replace(abbrev, word)
    tweet = re.sub(r'\b(http|https):\/\/.*[^ alt]\b', '', tweet)
    clean_tweets.append(tweet)
    
clean_tweets[:4]


['I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.',
 'I-10 East at Mesa, schedule road work, all lanes closed, follow detour, clearing time 6 am.',
 'I-10 East  at US-54, Collision, Right Lane Closed, Backup To Porfirio Diaz, Clearing Time 1 Hour.',
 'I-10 East  at Schuster, Collision, Right Shoulder Closed, Backup To Executive, Clearing Time 1 Hour.']

In [60]:
# Return only proper nouns from list of clean tweets
# nlp = spacy.load('en_core_web_sm')
# addresses = []
# for i, tweet in enumerate(clean_tweets):
#     doc = nlp(tweet)
#     address = ''
#     for token in doc:
#         if token.pos_ == 'PROPN':
#             address += str(token)
#             address += ' '
#     addresses.append(address)

# print(len(addresses))
# addresses[::20]

103


['I-10 East Airway ',
 'I-10 East Brown Collision Center Lane Closed Backup Clearing Time Hour ',
 'I-10 East Zaragoza Collision Backup Lee Trevino Clearing ',
 'Beaumont Winnie Harvey ',
 'SH Refugio County SH Tivoli Calhoun County ',
 'US Fry Barker Cypress ']

In [14]:
# Regex to get good locations from filtered locations above
locations = []

for ad in clean_tweets:
    match_1 = re.search("I\-[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", ad)
    match_2 = re.search("[0-9a-zA-Z]+\sand+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", ad)
    match_3 = re.search("I[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", ad)
    match_4 = re.search("[a-zA-Z]+\sat+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", ad)
    match_5 = re.search("I\-[0-9]+[0-9a-zA-Z]+\s@+[0-9a-zA-Z]+\s[0-9a-zA-Z]+", ad)
    
    if match_1: locations.append(match_1.group())
    elif match_2: locations.append(match_2.group())
    elif match_3: locations.append(match_3.group())
    elif match_4: locations.append(match_4.group())
    elif match_5: locations.append(match_5.group())
        
print(len(locations))
locations

43


['I-10 East at Airway',
 'I-10 East at Mesa',
 'I-10 East at Asarco',
 'I-10 West at Lomaland',
 'I-10 East at Sunland',
 'I-10 East at Sunland',
 'I-10 West at US',
 'I-10 West at Hawkins',
 'I-10 West at Sunland',
 'I-10 East at Mesa',
 'South at Spur 601',
 'I-10 West at Harnose',
 'I-10 West at Sunland',
 'I-10 East at Mesa',
 'custody at 1750 Lee',
 'I-10 West at Executive',
 'I-10 East at Mesa',
 'I-10 West at Porfirio',
 'I-10 East at Americas',
 'closed at Lakeshore boulevard',
 'crossing at 5003 Wasson',
 'I10 westbound from Neches',
 'Beaumont and Winnie is',
 'Beaumont and Winnie is',
 'closed and the water',
 'north and west on',
 'water at the foot',
 'Beaumont and Winnie now',
 'I10 westbound near SH146',
 'Magnolia at Twin City',
 'closed at Beason Creek',
 '1069 and FM 136',
 'closed at SH 239',
 'northbound at the San',
 'bridge and overturned heavy',
 'I-820 to Basswood boulevard',
 'I-35 northbound Exit Ramp',
 'I-35 northbound FR at',
 'I-35 northbound FR at',
 'I-3

# Create and Export Locations

In [16]:
locations_df = pd.DataFrame(locations, columns=['location'])
locations_df.shape

(43, 1)

In [17]:
locations_df.head()

,location
0,I-10 East at Airway
1,I-10 East at Mesa
2,I-10 East at Asarco
3,I-10 West at Lomaland
4,I-10 East at Sunland


In [19]:
locations_df.to_csv('./data/locations.csv', index=False)

# Unneeded

In [21]:
df['tweet'].tail()

2914    Waxahachie | US 287 Business WB@I-35 E | Various lanes closed today 9AM to 3PM http://i35-maps.tti.tamu.edu/?id=6342 #My35               
2948    ALERT: SL 287 East remains closed in Lufkin. Alternate routes are: SL 287 West, BU 59. From Nacogdoches, SH7 W to US 69 S to SL 287 West.
3040    US 290 EB closed from Fry to Barker Cypress is closed. Only high clearance emergency vehicles can get through.                           
3053    No, if you do not need to travel, please stay put. US 290 EB is closed at Fry. Visit http://drivetexas.org to see updates.               
3069    US 290 mainlanes & frontage are closed in both directions from Barker Cypress to Fry Road due to high water.                             
Name: tweet, dtype: object

In [22]:
locations[:-10]

['I-10 East at Airway',
 'I-10 East at Mesa',
 'I-10 East at Asarco',
 'I-10 West at Lomaland',
 'I-10 East at Sunland',
 'I-10 East at Sunland',
 'I-10 West at US',
 'I-10 West at Hawkins',
 'I-10 West at Sunland',
 'I-10 East at Mesa',
 'South at Spur 601',
 'I-10 West at Harnose',
 'I-10 West at Sunland',
 'I-10 East at Mesa',
 'custody at 1750 Lee',
 'I-10 West at Executive',
 'I-10 East at Mesa',
 'I-10 West at Porfirio',
 'I-10 East at Americas',
 'closed at Lakeshore boulevard',
 'crossing at 5003 Wasson',
 'I10 westbound from Neches',
 'Beaumont and Winnie is',
 'Beaumont and Winnie is',
 'closed and the water',
 'north and west on',
 'water at the foot',
 'Beaumont and Winnie now',
 'I10 westbound near SH146',
 'Magnolia at Twin City',
 'closed at Beason Creek',
 '1069 and FM 136',
 'closed at SH 239']

In [20]:
tweets = list()
for tweet in addresses:
    match_1 = re.search("I\-[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_2 = re.search("[0-9a-zA-Z]+\s@+[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_3 = re.search("[0-9a-zA-Z]+\sand+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_4 = re.search("I[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_5 = re.search("[a-zA-Z]+\sat+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_6 = re.search("I\-[0-9]+[0-9a-zA-Z]+\s@+[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    if match_1: tweets.append(match_1.group())
    elif match_2: tweets.append(match_2.group())
    elif match_3: tweets.append(match_3.group())
    elif match_4: tweets.append(match_4.group())
    elif match_5: tweets.append(match_5.group())
    elif match_6: tweets.append(match_6.group())

print(len(tweets))
tweets

34


['I-10 East Schuster Shoulder',
 'I-10 West Trowbridge Right',
 'I-10 East Kansas Collision',
 'I-10 West Zaragoza Backup',
 'I-10 East Asarco Executive',
 'I-10 West Lomaland Lee',
 'I-10 East Sunland Park',
 'I-10 East Sunland Park',
 'I-10 West Paisano Exit',
 'I-10 East Piedras Collision',
 'I-10 East Schuster Streetalled',
 'I-10 West Sunland Park',
 'I-10 East Brown Collision',
 'I-10 East Schuster Collision',
 'I-10 West Sunland Park',
 'I-10 East Americas Collision',
 'I-10 West Executive Center',
 'I-10 West Vinton Update',
 'I-10 West Raynolds Collision',
 'I-10 West Raynolds Collision',
 'I-10 West Vinton Landfill',
 'I-10 West Executive Center',
 'I-10 West Porfirio Diaz',
 'I-10 East Zaragoza Lee',
 'I-10 East Zaragoza Collision',
 'I-10 East US Collision',
 'I-10 East Resler Collision',
 'I-10 East Geronimo Paisano',
 'I-10 East Americas Loop',
 'I10 EAST WESTBOUND BMT',
 'I-35 S 57th Street',
 'I-35 FR Ira Young',
 'I-35 S 57th Street',
 'I-35 FR Ira Young']

In [21]:
tweets = list()
for tweet in addresses:
    match_1 = re.search("I\-[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_2 = re.search("[0-9a-zA-Z]+\s@+[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_3 = re.search("[0-9a-zA-Z]+\sand+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_4 = re.search("I[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_5 = re.search("[a-zA-Z]+\sat+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_6 = re.search("I\-[0-9]+[0-9a-zA-Z]+\s@+[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    if match_1: tweets.append(match_1.group())
    elif match_2: tweets.append(match_2.group())
    elif match_3: tweets.append(match_3.group())
    elif match_4: tweets.append(match_4.group())
    elif match_5: tweets.append(match_5.group())
    elif match_6: tweets.append(match_6.group())

print(len(tweets))
tweets

34


['I-10 East Schuster Shoulder',
 'I-10 West Trowbridge Right',
 'I-10 East Kansas Collision',
 'I-10 West Zaragoza Backup',
 'I-10 East Asarco Executive',
 'I-10 West Lomaland Lee',
 'I-10 East Sunland Park',
 'I-10 East Sunland Park',
 'I-10 West Paisano Exit',
 'I-10 East Piedras Collision',
 'I-10 East Schuster Streetalled',
 'I-10 West Sunland Park',
 'I-10 East Brown Collision',
 'I-10 East Schuster Collision',
 'I-10 West Sunland Park',
 'I-10 East Americas Collision',
 'I-10 West Executive Center',
 'I-10 West Vinton Update',
 'I-10 West Raynolds Collision',
 'I-10 West Raynolds Collision',
 'I-10 West Vinton Landfill',
 'I-10 West Executive Center',
 'I-10 West Porfirio Diaz',
 'I-10 East Zaragoza Lee',
 'I-10 East Zaragoza Collision',
 'I-10 East US Collision',
 'I-10 East Resler Collision',
 'I-10 East Geronimo Paisano',
 'I-10 East Americas Loop',
 'I10 EAST WESTBOUND BMT',
 'I-35 S 57th Street',
 'I-35 FR Ira Young',
 'I-35 S 57th Street',
 'I-35 FR Ira Young']

In [7]:
# Look at all tweets
df

,dates&time,user,tweet,class
239,2017-09-01 04:36:40+00:00,EPPOLICE,"I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.",1
240,2017-09-01 04:09:53+00:00,EPPOLICE,"I-10 East at Mesa, schedule road work, all lanes closed, follow detour, clearing time 6 am.",1
242,2017-09-01 00:39:43+00:00,EPPOLICE,"I-10 East @US-54, Collision, Right Lane Closed, Backup To Porfirio Diaz, Clearing Time 1 Hour.",1
247,2017-08-31 23:10:23+00:00,EPPOLICE,"I-10 East @Schuster, Collision, Right Shoulder Closed, Backup To Executive, Clearing Time 1 Hour.",1
248,2017-08-31 22:51:34+00:00,EPPOLICE,"I-10 West @Trowbridge, Collision, Right Lane Closed, Backup To Hawkins, Clearing Time 1 Hour.",1
250,2017-08-31 22:28:02+00:00,EPPOLICE,"I-10 East @Kansas, Collision, Center lane closed, Backup to Asarco, Clearing time 1 hr, Alt Rte",1
253,2017-08-31 16:18:28+00:00,EPPOLICE,"I-10 West @Zaragoza, Stalled vehicle, Left lane closed, Backup to Don Haskins, Clearing time 1 hr",1
256,2017-08-31 15:21:25+00:00,EPPOLICE,"I-10 East at Asarco, stalled vehicle, center lane closed, backup to Executive Center, clearing time 1 hr",1
258,2017-08-31 14:39:20+00:00,EPPOLICE,"I-10 West at Lomaland, collision, left two lanes closed, backup to Lee Trevino, clearing time 1 hr.",1
260,2017-08-31 14:16:54+00:00,EPPOLICE,"I-10 East at Sunland Park, collision, right lane closed, minor backup, clearing time 1 hr.",1


Up to row 868 above:

I-10 East at Asarco
I-10 West @Zaragoza
Sunland Park Drive at Paisano
I-10 East @Schuster
I-10 West at US-54
Loop 375 West @Transmountain
Loop 375 South at Spur 601
Summit St. at Memorial Park
6019 Spicewood Springs, Austin, TX
SB lanes of S. Pleasant Valley Rd. are closed at Lakeshore Blvd
5003 Wasson Rd
1700 BLK IH 35 NB 
I10 between Beaumont & Winnie
ORANGE CO– I10 westbound from Neches River-Sabine River

In [154]:
# Clean tweets
# Map common abbreviations to full-length word
abbrevs = {
    'SB': 'southbound',
    'NB': 'northbound', 
    'EB': 'eastbound', 
    'WB': 'westbound', 
    'St': 'Street', 
    'Rd': 'Road', 
    'E.': 'east',
    'S.': 'south',
    'W.': 'west',
    'N.': 'north',
    'Blvd': 'boulevard',
    'Pkwy': 'parkway',
    'Hwy': 'highway',
    '@': ' at '  
}

# Map 
clean_tweets = []
for tweet in df['tweet']:
    for abbrev, word in abbrevs.items():
        tweet = tweet.replace(abbrev, word)
    
    tweet = re.sub(r'\b(http|https):\/\/.*[^ alt]\b', '', tweet)
    for word in ['|', 'Collision' , 'Backup', 'Shoulder', 'Closed']:
        tweet = tweet.replace(word, '')
    clean_tweets.append(tweet)
    
clean_tweets[:4]


['I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.',
 'I-10 East at Mesa, schedule road work, all lanes closed, follow detour, clearing time 6 am.',
 'I-10 East  at US-54, , Right Lane Closed,  To Porfirio Diaz, Clearing Time 1 Hour.',
 'I-10 East  at Schuster, , Right  Closed,  To Executive, Clearing Time 1 Hour.']

In [155]:
# Return only proper nouns from list of clean tweets
nlp = spacy.load('en_core_web_sm')
addresses = []
for i, tweet in enumerate(clean_tweets):
    doc = nlp(tweet)
    address = ''
    for token in doc:
        if token.pos_ == 'PROPN':
            address += str(token)
            address += ' '
    addresses.append(address)

print(len(addresses))
addresses[::20]

103


['I-10 East Airway ',
 'I-10 East Brown Center Lane Closed Clearing Time Hour ',
 'I-10 East Zaragoza Lee Trevino Clearing ',
 'Beaumont Winnie Harvey ',
 'SH Refugio County SH Tivoli Calhoun County ',
 'US Fry Barker Cypress ']

In [156]:
# Return only first 4 elements of each to get rid of unnecessary 
short_addresses = []
for ad in addresses:
    ad_list = ad.split()
    ad = ' '.join(ad_list[:4])
    short_addresses.append(ad)

print(len(short_addresses))
short_addresses[::20]

103


['I-10 East Airway',
 'I-10 East Brown Center',
 'I-10 East Zaragoza Lee',
 'Beaumont Winnie Harvey',
 'SH Refugio County SH',
 'US Fry Barker Cypress']

In [157]:
# Add 

In [158]:
short_addresses

['I-10 East Airway',
 'I-10 East Mesa',
 'I-10 East US-54 Right',
 'I-10 East Schuster Closed',
 'I-10 West Trowbridge Right',
 'I-10 East Kansas Center',
 'I-10 West Zaragoza Don',
 'I-10 East Asarco Executive',
 'I-10 West Lomaland Lee',
 'I-10 East Sunland Park',
 'I-10 East Sunland Park',
 'Sunland Park Drive Paisano',
 'I-10 West Paisano Exit',
 'I-10 East Piedras Santa',
 'I-10 East Schuster Streetalled',
 'I-10 West US-54 US-54',
 'Loop Transmountain US Clearing',
 'I-10 West Hawkins',
 'I-10 West Sunland Park',
 'I-10 East Mesa',
 'I-10 East Brown Center',
 'I-10 East Schuster Schuster',
 'Loop South Spur',
 'I-10 West Harnose',
 'I-10 West Sunland Park',
 'I-10 East Mesa',
 'Loop Montwood',
 'Lee Trevino Crisis Management',
 'I-10 East Americas Zaragoza',
 'Loop East US US',
 'I-10 West Executive Center',
 'I-10 West Vinton Update',
 'I-10 West Raynolds Airway',
 'I-10 West Raynolds Airway',
 'I-10 West Vinton Landfill',
 'Loop Montwood',
 'I-10 West Executive Center',
 'I-10 

## Testing

In [ ]:
# Create list of Regex

In [ ]:
addresses = []
for tweet in df['tweet']:
    print(tweet.contains('', regex='I\-[0-9]+\s.[a-zA-Z]+\s[a-zA-Z]+\s[a-zA-Z]+'))
#     address = tweet.contains('', regex='I\-[0-9]+\s.[a-zA-Z]+\s[a-zA-Z]+\s[a-zA-Z]+')
#     addresses.append(address)
# addresses

In [21]:
mask = df['tweet'].str.contains('', regex='I\-[0-9]+\s.[a-zA-Z]+\s[a-zA-Z]+\s[a-zA-Z]+')
df[mask]

,dates&time,user,tweet,class
239,2017-09-01 04:36:40+00:00,EPPOLICE,"I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.",1
240,2017-09-01 04:09:53+00:00,EPPOLICE,"I-10 East at Mesa, schedule road work, all lanes closed, follow detour, clearing time 6 am.",1
242,2017-09-01 00:39:43+00:00,EPPOLICE,"I-10 East @US-54, Collision, Right Lane Closed, Backup To Porfirio Diaz, Clearing Time 1 Hour.",1
247,2017-08-31 23:10:23+00:00,EPPOLICE,"I-10 East @Schuster, Collision, Right Shoulder Closed, Backup To Executive, Clearing Time 1 Hour.",1
248,2017-08-31 22:51:34+00:00,EPPOLICE,"I-10 West @Trowbridge, Collision, Right Lane Closed, Backup To Hawkins, Clearing Time 1 Hour.",1
250,2017-08-31 22:28:02+00:00,EPPOLICE,"I-10 East @Kansas, Collision, Center lane closed, Backup to Asarco, Clearing time 1 hr, Alt Rte",1
253,2017-08-31 16:18:28+00:00,EPPOLICE,"I-10 West @Zaragoza, Stalled vehicle, Left lane closed, Backup to Don Haskins, Clearing time 1 hr",1
256,2017-08-31 15:21:25+00:00,EPPOLICE,"I-10 East at Asarco, stalled vehicle, center lane closed, backup to Executive Center, clearing time 1 hr",1
258,2017-08-31 14:39:20+00:00,EPPOLICE,"I-10 West at Lomaland, collision, left two lanes closed, backup to Lee Trevino, clearing time 1 hr.",1
260,2017-08-31 14:16:54+00:00,EPPOLICE,"I-10 East at Sunland Park, collision, right lane closed, minor backup, clearing time 1 hr.",1


In [ ]:
# From Song's notebook
mask1_1 = df1['tweet'].str.contains('roa', regex = False, case = False, na=False)
mask1_2 = df1['tweet'].str.contains('stree', regex = False, case = False, na=False)
mask1_3 = df1['tweet'].str.contains('highway', regex = False, case = False, na=False)
mask1_4 = df1['tweet'].str.contains('ramp', regex=False, case=False, na=False)
mask1_5 = df1['tweet'].str.contains('st', regex=False, case=False, na=False)
mask1_6 = df1['tweet'].str.contains('ave', regex=False, case=False, na=False)
mask1_7 = df1['tweet'].str.contains('lane', regex=False, case=False, na=False)
mask1_8 = df1['tweet'].str.contains('ln', regex=False, case=False, na=False)
mask1_9 = df1['tweet'].str.contains('drive', regex=False, case=False, na=False)
mask1_10 = df1['tweet'].str.contains('closed', regex=False, case=False,na=False)
mask1_11 = df1['tweet'].str.contains('blvd', regex=False, case=False,na=False)
mask1_12 = df1['tweet'].str.contains('boulevard', regex=False, case=False, na=False)
mask1_13 = df1['tweet'].str.contains('pkwy', regex=False, case=False, na=False)
mask1_14 = df1['tweet'].str.contains('parkway', regex=False, case=False, na=False)

In [ ]:
df3[df3["2"].str.contains("I.[0-9+]. [a-zA-Z]+ [a-zA-Z]+", regex=True)]

'I\-+[0-9+]. [a-zA-Z]+ [a-zA-Z]+'

'(I\-+[0-9+]. (at|and) [a-zA-Z]+)|([a-z]+)'

## Scraping from gps-coordinates

In [27]:
baseurl = 'https://www.gps-coordinates.net/'
res = requests.get(baseurl)
res.status_code

200

In [81]:
df['tweet']

239     I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.                                              
240     I-10 East at Mesa, schedule road work, all lanes closed, follow detour, clearing time 6 am.                                                 
242     I-10 East @US-54, Collision, Right Lane Closed, Backup To Porfirio Diaz, Clearing Time 1 Hour.                                              
247     I-10 East @Schuster, Collision, Right Shoulder Closed, Backup To Executive, Clearing Time 1 Hour.                                           
248     I-10 West @Trowbridge, Collision, Right Lane Closed, Backup To Hawkins, Clearing Time 1 Hour.                                               
250     I-10 East @Kansas, Collision, Center lane closed, Backup to Asarco, Clearing time 1 hr, Alt Rte                                             
253     I-10 West @Zaragoza, Stalled vehicle, Left lane closed, Backup to Don Haskins, Clearing time 1 hr 

In [118]:
nlp = spacy.load('en_core_web_sm')
addresses = []
for i, tweet in enumerate(clean_tweets):
    doc = nlp(tweet)
    address = ''
    for token in doc:
        if token.pos_ == 'PROPN':
            address += str(token)
            address += ' '
    addresses.append(address)

print(len(addresses))
addresses

103


['I-10 East Airway ',
 'I-10 East Mesa ',
 'I-10 East US-54 Collision Right Lane Closed Backup Porfirio Diaz Clearing Time Hour ',
 'I-10 East Schuster Shoulder Backup Executive Clearing Time Hour ',
 'I-10 West Trowbridge Right Lane Closed Backup Hawkins Clearing Time Hour ',
 'I-10 East Kansas Collision Center Backup Asarco Clearing Alt Rte ',
 'I-10 West Zaragoza Backup Don Haskins Clearing ',
 'I-10 East Asarco Executive Center ',
 'I-10 West Lomaland Lee Trevino ',
 'I-10 East Sunland Park ',
 'I-10 East Sunland Park Sunland Park ',
 'Sunland Park Drive Paisano Sunland Park Drive North Desert Paisano ',
 'I-10 West Paisano Exit 23B Exit Minor Backup ',
 'I-10 East Piedras Collision Shoulder Backup Santa Fe Clearing ',
 'I-10 East Schuster Streetalled Right Lane Closed Minor Backup Clearing Time Hour ',
 'I-10 West US-54 US-54 North ',
 'Loop Transmountain Collision Backup US Clearing ',
 'I-10 West Hawkins ',
 'I-10 West Sunland Park ',
 'I-10 East Mesa ',
 'I-10 East Brown Collis

In [120]:
len(short_addresses)

103

In [123]:
short_addresses_only = []
for ad in addresses:
    ad_list = ad.split()
    if len(ad_list) <= 4:
        short_addresses_only.append(ad)

len(short_addresses_only)

49

In [119]:
short_addresses = []
for ad in addresses:
    ad_list = ad.split()
    ad = ' '.join(ad_list[:4])
    short_addresses.append(ad)
short_addresses

['I-10 East Airway',
 'I-10 East Mesa',
 'I-10 East US-54 Collision',
 'I-10 East Schuster Shoulder',
 'I-10 West Trowbridge Right',
 'I-10 East Kansas Collision',
 'I-10 West Zaragoza Backup',
 'I-10 East Asarco Executive',
 'I-10 West Lomaland Lee',
 'I-10 East Sunland Park',
 'I-10 East Sunland Park',
 'Sunland Park Drive Paisano',
 'I-10 West Paisano Exit',
 'I-10 East Piedras Collision',
 'I-10 East Schuster Streetalled',
 'I-10 West US-54 US-54',
 'Loop Transmountain Collision Backup',
 'I-10 West Hawkins',
 'I-10 West Sunland Park',
 'I-10 East Mesa',
 'I-10 East Brown Collision',
 'I-10 East Schuster Collision',
 'Loop South Spur',
 'I-10 West Harnose',
 'I-10 West Sunland Park',
 'I-10 East Mesa',
 'Loop Montwood',
 'Lee Trevino Crisis Management',
 'I-10 East Americas Collision',
 'Loop East US US',
 'I-10 West Executive Center',
 'I-10 West Vinton Update',
 'I-10 West Raynolds Collision',
 'I-10 West Raynolds Collision',
 'I-10 West Vinton Landfill',
 'Loop Montwood',
 'I-10

In [29]:
tweets = list()
for tweet in df["tweet"]:
    match_1 = re.search("I\-[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_2 = re.search("[0-9a-zA-Z]+\s@+[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_3 = re.search("[0-9a-zA-Z]+\sand+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_4 = re.search("I[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_5 = re.search("[a-zA-Z]+\sat+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
#     match_6 = re.search("I\-[0-9]+[0-9a-zA-Z]+\s@+[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    if match_1: tweets.append(match_1.group())
    elif match_2: tweets.append(match_2.group())
    elif match_3: tweets.append(match_3.group())
    elif match_4: tweets.append(match_4.group())
    elif match_5: tweets.append(match_5.group())
#     elif match_6: tweets.append(match_6.group())

len(tweets)

34

In [25]:
tweets = list()
for tweet in df['tweet']:
    match_1 = re.search("I\-[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_2 = re.search("[0-9a-zA-Z]+\s@+[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_3 = re.search("[0-9a-zA-Z]+\sand+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_4 = re.search("I[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_5 = re.search("[a-zA-Z]+\sat+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_6 = re.search("I\-[0-9]+[0-9a-zA-Z]+\s@+[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    if match_1: tweets.append(match_1.group())
    elif match_2: tweets.append(match_2.group())
    elif match_3: tweets.append(match_3.group())
    elif match_4: tweets.append(match_4.group())
    elif match_5: tweets.append(match_5.group())
    elif match_6: tweets.append(match_6.group())

print(len(tweets))
tweets

43


['I-10 East at Airway',
 'I-10 East at Mesa',
 'I-10 East at Asarco',
 'I-10 West at Lomaland',
 'I-10 East at Sunland',
 'I-10 East at Sunland',
 'West @Paisano Exit',
 'I-10 West at US',
 'I-10 West at Hawkins',
 'I-10 West at Sunland',
 'I-10 East at Mesa',
 'South at Spur 601',
 'I-10 West at Harnose',
 'I-10 West at Sunland',
 'I-10 East at Mesa',
 'custody at 1750 Lee',
 'East @US 54',
 'West @Executive Center',
 'I-10 West at Executive',
 'I-10 East at Mesa',
 'I-10 West at Porfirio',
 'East @US 54',
 'I-10 East at Americas',
 'closed at Lakeshore Blvd',
 'crossing at 5003 Wasson',
 'I10 westbound from Neches',
 'Beaumont and Winnie is',
 'Beaumont and Winnie is',
 'closed and the water',
 'westbound @state line',
 'water at the foot',
 'Beaumont and Winnie now',
 'I10 westbound near SH146',
 'Magnolia at Twin City',
 'closed at Beason Creek',
 '1069 and FM 136',
 'closed at SH 239',
 'NB at the San',
 'bridge and overturned heavy',
 'I-820 to Basswood Blvd',
 'I-35 NB Exit Ramp

In [ ]:
['I-10 East at Airway',
 'I-10 East at Mesa',
 'I-10 East at Asarco',
 'I-10 West at Lomaland',
 'I-10 East at Sunland',
 'I-10 East at Sunland',
 'West @Paisano Exit',
 'I-10 West at US',
 'I-10 West at Hawkins',
 'I-10 West at Sunland',
 'I-10 East at Mesa',
 'South at Spur 601',
 'I-10 West at Harnose',
 'I-10 West at Sunland',
 'I-10 East at Mesa',
 'custody at 1750 Lee',
 'East @US 54',
 'West @Executive Center',
 'I-10 West at Executive',
 'I-10 East at Mesa',
 'I-10 West at Porfirio',
 'East @US 54',
 'I-10 East at Americas',
 'closed at Lakeshore Blvd',
 'crossing at 5003 Wasson',
 'I10 westbound from Neches',
 'Beaumont and Winnie is',
 'Beaumont and Winnie is',
 'closed and the water',
 'westbound @state line',
 'water at the foot',
 'Beaumont and Winnie now',
 'I10 westbound near SH146',
 'Magnolia at Twin City',
 'closed at Beason Creek',
 '1069 and FM 136',
 'closed at SH 239',
 'NB at the San',
 'bridge and overturned heavy',
 'I-820 to Basswood Blvd',
 'I-35 NB Exit Ramp',
 'I-35 NB Exit Ramp',
 'I-820 to Basswood Blvd']

In [78]:
nlp = spacy.load('en_core_web_sm')
doc = nlp("I-10 East @US-54, Collision, Right Lane Closed, Backup To Porfirio Diaz, Clearing Time 1 Hour.  ")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

I-10 I-10 PROPN NNP compound X-dd False False
East East PROPN NNP compound Xxxx True False
@US-54 @us-54 NOUN NN nsubj @XX-dd False False
, , PUNCT , punct , False False
Collision collision NOUN NN appos Xxxxx True False
, , PUNCT , punct , False False
Right Right PROPN NNP compound Xxxxx True False
Lane Lane PROPN NNP compound Xxxx True False
Closed Closed PROPN NNP ROOT Xxxxx True False
, , PUNCT , punct , False False
Backup Backup PROPN NNP npadvmod Xxxxx True False
To to ADP IN prep Xx True True
Porfirio Porfirio PROPN NNP compound Xxxxx True False
Diaz Diaz PROPN NNP pobj Xxxx True False
, , PUNCT , punct , False False
Clearing Clearing PROPN NNP compound Xxxxx True False
Time Time PROPN NNP npadvmod Xxxx True False
1 1 NUM CD nummod d False False
Hour Hour PROPN NNP appos Xxxx True False
. . PUNCT . punct . False False
    SPACE _SP    False False


In [77]:
# MAYBE USE - in combo with earlier
nlp = spacy.load('en_core_web_sm')
doc = nlp("I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.")

for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

Airway Airway pobj at
all lanes lanes nsubj closed
detour detour dobj follow
clearing clearing conj follow
time time dobj clearing
5 am am dobj clearing


In [74]:
# MAYBE USE - in combo with earlier
nlp = spacy.load('en_core_web_sm')
doc = nlp("I-10 East @US-54, Collision, Right Lane Closed, Backup To Porfirio Diaz, Clearing Time 1 Hour.  ")

for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

I-10 East @US-54 @US-54 nsubj Closed
Collision Collision appos @US-54
Porfirio Diaz Diaz pobj To
1 Hour Hour appos Time


In [76]:
nlp = spacy.load('en_core_web_sm')
addresses = []
for i, tweet in enumerate(df['tweet']):
    doc = nlp(tweet)
    address = ''
    for token, chunk in zip(doc, doc.noun_chunks):
        if (token.pos_ == 'PROPN') & (chunk.root.dep_ == 'nsubj'):
            address += str(chunk)
            address += ' '
    addresses.append(address)

print(len(addresses))
addresses

103


['all lanes ',
 'all lanes ',
 'I-10 East @US-54 ',
 'Right Shoulder ',
 '',
 'East @Kansas Rte ',
 'I-10 West @Zaragoza Left lane ',
 'stalled vehicle, center lane ',
 '',
 'right lane ',
 'Sunland Park ',
 '',
 '',
 'East @Piedras ',
 '',
 'right lane ',
 'All main lanes ',
 'all lanes ',
 '',
 'all lanes ',
 '',
 'Schuster ',
 'Loop ',
 'right shoulder ',
 '',
 'all lanes ',
 '',
 '',
 'East @Americas ',
 'US 54 north ramp N ',
 'West @Executive Center Left lane ',
 '',
 'Right lane ',
 'Right lane ',
 'West @Vinton ',
 '',
 '',
 'all lanes ',
 '',
 'East @Zaragoza ',
 'East @Zaragoza ',
 'Collision Right two lanes ',
 '',
 'Right lane ',
 'I-10 west @Americas ',
 'all lanes ',
 'all lanes ',
 'all lanes ',
 'exit ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'ORANGE CO ',
 '',
 '',
 '',
 '',
 '',
 'US69 ',
 'Louisiana State Police ',
 '',
 'Rainbow Bridge STAY ',
 '',
 'SH73 ',
 '',
 'I10 EAST ',
 '',
 '',
 '',
 '',
 'FM ',
 '',
 '',
 '',
 'SH6 northbound ',
 '',
 '',
 'TxDOT ',
 'US 59/ 

In [9]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

Autonomous cars cars nsubj shift
insurance liability liability dobj shift
manufacturers manufacturers pobj toward


In [10]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.")
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

Airway Airway pobj at
all lanes lanes nsubj closed
detour detour dobj follow
clearing clearing conj follow
time time dobj clearing
5 am am dobj clearing


In [15]:
import numpy as np
from spacy.attrs import ENT_IOB, ENT_TYPE

In [17]:
nlp = spacy.load("en_core_web_sm")
doc = nlp.make_doc("London is a big city in the United Kingdom.")
print("Before", doc.ents)  # []

header = [ENT_IOB, ENT_TYPE]
attr_array = np.zeros((len(doc), len(header)))
attr_array[0, 0] = 3  # B
attr_array[0, 1] = doc.vocab.strings["GPE"]
doc.from_array(header, attr_array)
print("After", doc.ents)  # [London]

Before ()
After (London,)


In [19]:
nlp = spacy.load("en_core_web_sm")
doc = nlp.make_doc("I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.")
print("Before", doc.ents)  # []

header = [ENT_IOB, ENT_TYPE]
attr_array = np.zeros((len(doc), len(header)))
attr_array[0, 0] = 3  # B
attr_array[0, 1] = doc.vocab.strings["GPE"]
doc.from_array(header, attr_array)
print("After", doc.ents)  # [London]

Before ()
After (I-10,)


In [24]:
from spacy import displacy

text = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
displacy.render(doc, style="ent")


In [25]:
nlp = spacy.load("my_custom_el_model")
doc = nlp("Ada Lovelace was born in London")

# document level
ents = [(e.text, e.label_, e.kb_id_) for e in doc.ents]
print(ents)  # [('Ada Lovelace', 'PERSON', 'Q7259'), ('London', 'GPE', 'Q84')]

# token level
ent_ada_0 = [doc[0].text, doc[0].ent_type_, doc[0].ent_kb_id_]
ent_ada_1 = [doc[1].text, doc[1].ent_type_, doc[1].ent_kb_id_]
ent_london_5 = [doc[5].text, doc[5].ent_type_, doc[5].ent_kb_id_]
print(ent_ada_0)  # ['Ada', 'PERSON', 'Q7259']
print(ent_ada_1)  # ['Lovelace', 'PERSON', 'Q7259']
print(ent_london_5)  # ['London', 'GPE', 'Q84']

OSError: [E050] Can't find model 'my_custom_el_model'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

## Regex w/ Spacy

In [27]:
import re
tweets = list()
for tweet in df["tweet"]:
    match_1 = re.search("I\-[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_2 = re.search("[0-9a-zA-Z]+\s@+[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_3 = re.search("[0-9a-zA-Z]+\sand+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_4 = re.search("I[0-9]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_5 = re.search("[a-zA-Z]+\sat+\s[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    match_6 = re.search("I\-[0-9]+[0-9a-zA-Z]+\s@+[0-9a-zA-Z]+\s[0-9a-zA-Z]+", tweet)
    if match_1: tweets.append(match_1.group())
    elif match_2: tweets.append(match_2.group())
    elif match_3: tweets.append(match_3.group())
    elif match_4: tweets.append(match_4.group())
    elif match_5: tweets.append(match_5.group())
    elif match_6: tweets.append(match_6.group())


In [28]:
tweets

['I-10 East at Airway',
 'I-10 East at Mesa',
 'I-10 East at Asarco',
 'I-10 West at Lomaland',
 'I-10 East at Sunland',
 'I-10 East at Sunland',
 'West @Paisano Exit',
 'I-10 West at US',
 'I-10 West at Hawkins',
 'I-10 West at Sunland',
 'I-10 East at Mesa',
 'South at Spur 601',
 'I-10 West at Harnose',
 'I-10 West at Sunland',
 'I-10 East at Mesa',
 'custody at 1750 Lee',
 'East @US 54',
 'West @Executive Center',
 'I-10 West at Executive',
 'I-10 East at Mesa',
 'I-10 West at Porfirio',
 'East @US 54',
 'I-10 East at Americas',
 'closed at Lakeshore Blvd',
 'crossing at 5003 Wasson',
 'I10 westbound from Neches',
 'Beaumont and Winnie is',
 'Beaumont and Winnie is',
 'closed and the water',
 'westbound @state line',
 'water at the foot',
 'Beaumont and Winnie now',
 'I10 westbound near SH146',
 'Magnolia at Twin City',
 'closed at Beason Creek',
 '1069 and FM 136',
 'closed at SH 239',
 'NB at the San',
 'bridge and overturned heavy',
 'I-820 to Basswood Blvd',
 'I-35 NB Exit Ramp

In [84]:
introduction_text = 'I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.'
doc = nlp(introduction_text)
for ent in doc.ents:
    print(ent, ent.label_)

Airway ORG
5 am TIME


In [30]:
introduction_text = ('I-10 East at Airway, scheduled road work, all lanes closed, follow detour, clearing time 5 am.')
introduction_doc = nlp(introduction_text)
print ([token.text for token in introduction_doc if token.label_ == 'GPE'])

AttributeError: 'spacy.tokens.token.Token' object has no attribute 'label_'